In [1]:
import tensorflow as tf
sess = tf.InteractiveSession()

In [2]:
def get_all_variables_from_top_scope(scope):
    #scope is a top scope here, otherwise change startswith part
    return [v for v in tf.all_variables() if v.name.startswith(scope.name)]

def create_model(tf, param1, param2, add=0):
    w = tf.get_variable('W', shape=[1], initializer=tf.constant_initializer(param1))
    w += add
    b = tf.get_variable('b', shape=[1], initializer=tf.constant_initializer(param2))
    y = tf.mul(w, b, name='mul_op')#no need to save this
    return y

In [3]:
import tensorflow as tf

import melt

OLD_SCOPE_NAME = 'scope1'
NEW_SCOPE_NAME = 'scope2'

sess = tf.Session()

#melt.reuse_variables()
with tf.variable_scope(OLD_SCOPE_NAME) as topscope:
    model = create_model(tf, 8.0, 7.0, 100)
    sc_vars = get_all_variables_from_top_scope(topscope)

print([v.name for v in sc_vars])
print([v.name for v in tf.all_variables()])
print(tf.all_variables())

saver = tf.train.Saver()
saver.restore(sess, '/tmp/%s'%OLD_SCOPE_NAME)
print(sess.run(model))


#assuming that we change top scope, not something in the middle, functionality can be added without much trouble I think
#not sure why I need to remove ':0' part, but it seems to work okay
print([NEW_SCOPE_NAME + v.name[len(OLD_SCOPE_NAME):v.name.rfind(':')] for v in sc_vars])
var_list = {NEW_SCOPE_NAME + v.name[len(OLD_SCOPE_NAME):v.name.rfind(':')]:v for v in sc_vars}

print(var_list)
print([v.name for v in tf.all_variables()])
print(tf.all_variables())

new_saver = tf.train.Saver(var_list=var_list)

#l = [NEW_SCOPE_NAME + v.name[len(OLD_SCOPE_NAME):v.name.rfind(':')] for v in sc_vars]
#new_saver.export_meta_graph('/tmp/%s.2'%NEW_SCOPE_NAME, collection_list=l)
#new_saver = tf.train.Saver()
new_saver.save(sess,  '/tmp/%s'%NEW_SCOPE_NAME)

[u'scope1/W:0', u'scope1/b:0']
[u'scope1/W:0', u'scope1/b:0']
[<tensorflow.python.ops.variables.Variable object at 0x6e486d0>, <tensorflow.python.ops.variables.Variable object at 0x6e46e10>]
[ 972.]
[u'scope2/W', u'scope2/b']
{u'scope2/W': <tensorflow.python.ops.variables.Variable object at 0x6e486d0>, u'scope2/b': <tensorflow.python.ops.variables.Variable object at 0x6e46e10>}
[u'scope1/W:0', u'scope1/b:0']
[<tensorflow.python.ops.variables.Variable object at 0x6e486d0>, <tensorflow.python.ops.variables.Variable object at 0x6e46e10>]


'/tmp/scope2'